In [ ]:
import mne
from scipy.stats import pearsonr
import numpy as np

def resample_corr(bc_orig, nk_orig, resample_base, resample_error, window_size, channel, print_corr=False):

    bc = bc_orig[channel]
    nk = nk_orig[channel]

    tot_len = len(bc)

    new_bc = [[] for _ in range(len(bc_orig))]

    lag = 0
    for i in range(0,tot_len,window_size):
        if i+window_size>tot_len:
            break
        bc_part = bc[i:i+window_size]
        max_corr = 0
        for resample_val in range(resample_base-resample_error, resample_base+resample_error):
            x = np.linspace(0, 1, len(bc_part))
            x_new = np.linspace(0, 1, resample_val)
            bc_resampled = np.interp(x_new, x, bc_part)
            nk_part = nk[lag:lag+resample_val]
            if len(nk_part) < len(bc_resampled):
                break
            corr = pearsonr(bc_resampled, nk_part)[0]
            if corr > max_corr:
                max_corr = corr
                resample_final = resample_val
                x_newf = x_new
                x_f = x
        lag += resample_final
        if print_corr:
            print('Samples:',resample_final,'Correlation:', max_corr)
        for j in range(len(bc_orig)):
            bc_resampled = np.interp(x_newf, x_f, bc_orig[j][i:i+window_size]).tolist()
            new_bc[j] += bc_resampled
    
    new_nk = nk_orig[:,:len(new_bc[0])]
    return new_bc, new_nk

# record - start/end - nk - bc
# 2 - s - 11157 - 2208
# 2 - e - 63904 - 27604

# 3 - s - 67306 - 1634
# 3 - e - 183164 - 57400

# 4 - s - 186112 - 1340
# 4 - e - 322130 - 66689

# 5 - s - 508346 - 1459
# 5 - e - 686371 - 87190
# %matplotlib widget

raw = mne.io.read_raw_nihon('EEG2101\FA0012E5.EEG', preload=True)
onsets_closed =[]
onsets_open = []
for i in raw.annotations:
    if i['description']=='Eyes closed':
        onsets_closed+=[i['onset']]
    if i['description']=='Eyes open':
        onsets_open+=[i['onset']]
onsets_closed = np.array(onsets_closed, dtype=int)*500
onsets_open = np.array(onsets_open, dtype=int)*500

raw.set_eeg_reference(['Cz']).pick(['T4','Fp2','C4','O2','O1','Fp1','C3','T3'])
raw = raw[:]
x = raw[1]
nk_orig = raw[0]*1000000
onsets_closed_ = np.zeros(len(nk_orig[0]))
onsets_closed_[onsets_closed] = 1
onsets_open_ = np.zeros(len(nk_orig[0]))
onsets_open_[onsets_open] = 1

nk = [[] for _ in range(4)]

nk[0] = nk_orig[:,11157:63904]
nk[1] = nk_orig[:,67306:183164]
nk[2] = nk_orig[:,186112:322130]
nk[3] = nk_orig[:,508346:686371]

on_c = [[] for _ in range(4)]

on_c[0] = onsets_closed_[11157:63904]
on_c[1] = onsets_closed_[67306:183164]
on_c[2] = onsets_closed_[186112:322130]
on_c[3] = onsets_closed_[508346:686371]

on_o = [[] for _ in range(4)]
on_o[0] = onsets_open_[11157:63904]
on_o[1] = onsets_open_[67306:183164]
on_o[2] = onsets_open_[186112:322130]
on_o[3] = onsets_open_[508346:686371]

bc = [[] for _ in range(4)]

bc[0] = np.genfromtxt('day2\Recording-20240318-145932.csv', delimiter=',')
bc[0] = bc[0][1:-1,12:20].T
bc[0] = bc[0][:,2208:27604]*1000

bc[1] = np.genfromtxt('day2\Recording-20240318-150211.csv', delimiter=',')
bc[1] = bc[1][1:-1,12:20].T
bc[1] = bc[1][:,1634:57400]*1000

bc[2] = np.genfromtxt('day2\Recording-20240318-150625.csv', delimiter=',')
bc[2] = bc[2][1:-1,12:20].T
bc[2] = bc[2][:,1340:66689]*1000

bc[3] = np.genfromtxt('day2\Recording-20240318-151801.csv', delimiter=',')
bc[3] = bc[3][1:-1,12:20].T
bc[3] = bc[3][:,1459:87190]*1000

for i in range(4):
    bc[i], nk[i] = resample_corr(bc[i], nk[i], 500, 100, 250, 4)
    on_o[i] = on_o[i][:len(nk[i][0])]
    on_c[i] = on_c[i][:len(nk[i][0])]
    on_o[i] = np.argwhere(on_o[i]==1).flatten()
    on_c[i] = np.argwhere(on_c[i]==1).flatten()

    np.save('day2/bc'+str(i), np.array(bc[i]))
    np.save('day2/nk'+str(i), np.array(nk[i]))
    np.save('day2/on_o'+str(i), on_o[i])
    np.save('day2/on_c'+str(i), on_c[i])
